In [189]:
from bs4 import BeautifulSoup
import re
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
import random
import os
import operator
import xml.dom.minidom
import numpy as np
import math
#from sets import Set
#from html.parser import HTMLParser

In [190]:
def get_directory_path(mode):
    """
    It takes only path of folder, no file name.
    It only returns the folder which contain all the text file.
    
    Argument:
    #nothing
    
    Returns:
    dp -- directory path which contains all the txt files.
    """
    if (mode == "input"):   
        dp = "/Users/imbilalbutt/Documents/Semesters/Semester 9/Information Retrieval/Assignment/hw2/input"
    elif (mode == "output"):
        dp = "/Users/imbilalbutt/Documents/Semesters/Semester 9/Information Retrieval/Assignment/hw2/out"
    else:
        raise ValueError('Unspecified mode for I/O.')
        dp = None

    return dp

In [191]:
# Function : read_stop_list
def read_text_in_list_form(file_path):
    """
    This function takes the path of stop words file and reads it and returns a list of words.
    
    Argument:
    stop_file_path -- path should be like: path + file name.extension
        "/Users/imbilalbutt/Documents/Semesters/Semester 9/Information Retrieval/Assignment/stoplist.txt".
    
    Returns:
    lineList -- list of words containg all the stop_words.
    """
    
    lst = [line.rstrip('\n') for line in open(file_path)]
    return lst

In [192]:
stop_word_file = "/Users/imbilalbutt/Documents/Semesters/Semester 9/Information Retrieval/Assignment/hw2/input/stoplist.txt"
stop_words = read_text_in_list_form(stop_word_file)

In [193]:
# Function : To open term_ids with terms
def open_inverted_index(file_name):
    tokensDict = dict()
    i = 0
    file = open(file_name,'r',encoding = "utf-8")
    for value in file:
        value = value.split("/")
        value[1] = value[1].strip("\n")
        tokensDict[str(value[1])] = str(value[0])
        i = i + 1
    file.close()
    return tokensDict

In [194]:
# def xml_parser(file_name):
#     doc = xml.dom.minidom.parse(file_name)
#     qrys = doc.getElementsByTagName('query')
#     tpcs = doc.getElementsByTagName('topic')
#     queries_list = list()
#     #qrys = mydoc.getElementsByTagName('topic')
#     i = 0
#     for elem in qrys:
#         #print(tpcs[i].attributes['number'].value)
#         queries_list.append(elem.firstChild.data)
#         i = i + 1
#         #print(elem.firstChild.data)
#     #print(queries_list)

## Version 2.0 using dictionary
def xml_parser(file_name):
    doc = xml.dom.minidom.parse(file_name)
    qrys = doc.getElementsByTagName('query')
    tpcs = doc.getElementsByTagName('topic')
    queries = dict()
    #qrys = mydoc.getElementsByTagName('topic')
    i = 0
    for elem in qrys:
        queries[tpcs[i].attributes['number'].value] = elem.firstChild.data
        #queries_list.append(elem.firstChild.data)
        i = i + 1
        #print(elem.firstChild.data)
    #print(queries)
    return queries

In [195]:
file_name = "/Users/imbilalbutt/Documents/Semesters/Semester 9/Information Retrieval/Assignment/hw2/input/topics.xml"
qrys = xml_parser(file_name)
#print(qrys[str(202)])
print(qrys)

{'202': 'uss carl vinson', '214': 'capital gains tax rate', '216': 'nicolas cage movies', '221': 'electoral college 2008 results', '227': 'i will survive lyrics', '230': "world's biggest dog", '234': 'dark chocolate health benefits', '243': 'afghanistan flag', '246': 'civil war battles in South Carolina', '250': 'ford edge problems'}


In [196]:
# def read_relevance_judgements(file_name):
#     file = open(file_name,'r',encoding = "utf-8")
#     for line in file:
#         splited = line.split()
#         #print(type(x))
#         #print(x)
    
        
        
#         #print(file_line)
    
    
# #     print(file_line)
# #     print(file_line[0])
# #     print(file_line[1])
# #     print(file_line[2])
# #     print(file_line[3])
# #     file_line = file.readline()
# #     print(file_line)
    
    

In [197]:
# file_name = "/Users/imbilalbutt/Documents/Semesters/Semester 9/Information Retrieval/Assignment/hw2/input/relevance judgements.qrel"
# read_relevance_judgements(file_name)

# Query Processing

In [198]:
def stem_words(tokenized_words_without_stop_words):
    """
    This function takes in list of words which do not contain stop_words.
    It uses the PorterStemmer() to reduce the word to their root words.
    
    Argument:
    removed_all_stop_words -- list of all words which do not have stop_words.
    
    Returns:
    stemmed_words -- list of words which are reduced to their origin word.
    """
    ps = PorterStemmer()
    stemmed_words = list()
    for w in tokenized_words_without_stop_words:
        stemmed_words.append(ps.stem(w))
    stemmed_words.sort()
    return stemmed_words

In [199]:
def query_processing(query_string):
    #splited_query = list(re.split(query))
    splited_query = list(query_string.split())
    #splited_query.lower()
    cleaned_tokens_from_stop_words =  list(set(splited_query) - set(stop_words))
    stemmed_tokens_of_words = stem_words(cleaned_tokens_from_stop_words)
    return stemmed_tokens_of_words

In [200]:
#print(type(qrys[str(202)]))
x = query_processing(qrys[str(202)])
print(len(x))

3


# Scoring Function 1: Okapi BM25

In [201]:
def get_all_documents_length(file_name):
    doc_lengths = dict()
    i = 0
    file = open(file_name,'r',encoding = "utf-8")
    
    for each_line in file:
        x = each_line.split()
        doc_lengths[x[0]] = x[2]
#        doc_frequency[each_line]
        #print(x[0])
    return doc_lengths 
    

In [202]:
doc_length_file = "/Users/imbilalbutt/Documents/Semesters/Semester 9/Information Retrieval/Assignment/hw2/input/docid_hashed.txt"
all_doc_lengths = get_all_documents_length(doc_length_file)
c = int(all_doc_lengths[str(3058)])
#print(int(all_doc_lengths[str(3058)]))
print(type(c))

<class 'int'>


In [203]:
def get_all_vocablury(file_name):
    vocablury = dict()
    file = open(file_name,'r',encoding = "utf-8")
    for each_line in file:
        x = each_line.split()
        #Interesting: making term as key and term_id as value (;
        vocablury[x[1]] = x[0]
    return vocablury

In [204]:
voc_file = "/Users/imbilalbutt/Documents/Semesters/Semester 9/Information Retrieval/Assignment/hw2/input/termid_hashed.txt"
vocablury = get_all_vocablury(voc_file)
#print(vocablury)

In [205]:
def get_document_postings(file_name):
    doc_freq = dict()
    file = open(file_name,'r',encoding = "utf-8")
    #print("to")
    for each_line in file:
        lst = list()
        x = each_line.split('\t')
        lineList = [line.rstrip('\n') for line in x]
        
#         z = x[2].strip("\n")
#         x = z.strip("[]")
#         lst = x.split(",")
        
        doc_freq[x[0]] = lineList
        #print((lineList[0]))
        #print((lineList[2]))
    return doc_freq
        
# def iter_list(x, new_arr):
#     if(isinstance(x,list)):
#         iter_list(x, new_arr)
#     else:
#         return new_arr.append(x)
            
       
    
    
#     with open(file_name) as f:
#         return map(int, f)

    



# #        print(x)
        #y = list(x[2])
        #lst = list()
        #for e in len(y):
            #lst.append(y[e])
        #doc_freq[x[0]] = lst
    
#     output = []
#     with open(file_name, 'r') as f:
#         w, h  = map(int, f.readline().split())
#         tmp = []
#         for i, line in enumerate(f):
#             if i == h:
#                 break
#             tmp.append(map(int, line.split()[:w]))
#         output.append(tmp)
#     with open(file_name) as f:
#         w , h = [int(x) for x in next(f).split()] # read first line
#         array = []
#         for line in f: # read rest of lines
#             array.append([int(x) for x in line.split()])

        
        
#         splitTextAlp = str(re.findall(r"[0-9]+", each_line))
#         splitTextAlp = list(re.split(r'\W+', (splitTextAlp)))
#         print(splitTextAlp)

        
#         x = each_line.split('\t')
#         print(type(x))


        #print(x[0])
        #print(x[1])
        
    
        #print(y)
        #y = x[2].strip("\n")
        #z = y.split(',')
        #print(type(x[2]))
        #lineList = [line.rstrip('\n') for line in x]
        #y = x[2].strip("\n")
        
    


In [206]:
doc_postings_file = "/Users/imbilalbutt/Documents/Semesters/Semester 9/Information Retrieval/Assignment/hw2/input/document_postings.txt"
dict_termid_with_docs_postings = get_document_postings(doc_postings_file)
#print(dict_termid_with_docs_postings)

#print(dict_termid_with_docs_postings)
for kys, values in dict_termid_with_docs_postings.items():
    #print( dict_termid_with_docs_postings[kys])
    c = dict_termid_with_docs_postings[kys]
    z = c[2].strip("\n")
    x = z.strip("[]")
    s = x.split(",")
    #print((s[0]))



In [207]:
def get_term_frequency(file_name):
    term_freq = dict()
    file = open(file_name,'r',encoding = "utf-8")
    for each_line in file:
        x = each_line.split("\t")
        term_freq[x[0]] = x[2]
    return term_freq

In [208]:
hashed_term_id = "/Users/imbilalbutt/Documents/Semesters/Semester 9/Information Retrieval/Assignment/hw2/input/term_index_hashed.txt"
dict_term_id_with_frequencies = get_term_frequency(hashed_term_id)
#print(dict_term_id_with_frequencies)

In [209]:
#print(df[str(551445)])

In [248]:
def calculate_okapi_bm25():
    
    k1 = 1.2 
    k2 = 800
    b = 0.75
    D = 17
    tf_q_i = 1
    
    queries_file_name = "/Users/imbilalbutt/Documents/Semesters/Semester 9/Information Retrieval/Assignment/hw2/input/topics.xml"
    queries_dict = xml_parser(queries_file_name) #returned -> dict
    
    doc_info_file = "/Users/imbilalbutt/Documents/Semesters/Semester 9/Information Retrieval/Assignment/hw2/input/docid_hashed.txt"
    all_doc_lengths = get_all_documents_length(doc_info_file)
    
    voc_file = "/Users/imbilalbutt/Documents/Semesters/Semester 9/Information Retrieval/Assignment/hw2/input/termid_hashed.txt"
    vocablury = get_all_vocablury(voc_file)
    
    doc_postings_file = "/Users/imbilalbutt/Documents/Semesters/Semester 9/Information Retrieval/Assignment/hw2/input/document_postings.txt"
    dict_termid_with_docs_postings = get_document_postings(doc_postings_file)
    
    cleared_dict_termid_with_docs_postings = dict()
    
    for kys, values in dict_termid_with_docs_postings.items():
    #print( dict_termid_with_docs_postings[kys])
        c = dict_termid_with_docs_postings[kys]
        z = c[2].strip("\n")
        x = z.strip("[]")
        s = x.split(",")
        cleared_dict_termid_with_docs_postings[kys] = s
    

    hashed_term_id = "/Users/imbilalbutt/Documents/Semesters/Semester 9/Information Retrieval/Assignment/hw2/input/term_index_hashed.txt"
    dict_term_id_with_frequencies = get_term_frequency(hashed_term_id)
    
    avg_length = 0
    for doc_id, length in all_doc_lengths.items():
        avg_length += int(length)
        #print((avg_length))
    avg_length /= len(all_doc_lengths) 
       
    scores_dictionary = dict()    
    #capital_K = k1 * ((1-b) + (b * (len(d)/avgr_d)))
    for query_id, query in queries_dict.items(): # run for 10 times
        splitted_query = query_processing(query)
        # Ab wo documents aa jaien jin mein ye term id hai.
        # split kerne k bad, har term ka doc k sath score nikalna hai
        score = 0
        # query ki eik term, phr dosri, phr teesri and so on.
        for i in range(0,len(splitted_query)):
            #print(splitted_query[i])
            # eik term pakar li
            # ab dekhna hai wo term kitne docs mein mujood hai
            #lkn pehle us term ka term_id le leya jae
            docs_score_for_each_query = dict()
            if splitted_query[i] in vocablury: # if that term exists
                #then get its term_id
                term_id = vocablury[splitted_query[i]]
                # ab dekhna hai ye term ktne docs mein mujood hai
                #print(term_id)
                list_of_all_docs_in_which_term_exists = cleared_dict_termid_with_docs_postings[str(term_id)]
                #print((list_of_all_docs_in_which_term_exists))
                #print(len(list_of_all_docs_in_which_term_exists))
                df_i = len(list_of_all_docs_in_which_term_exists)
                #tf_q_i = 1
                tf_d_i = int(dict_term_id_with_frequencies[str(term_id)])
                for i in range(0,len(list_of_all_docs_in_which_term_exists)):
                    doc_id = list_of_all_docs_in_which_term_exists[i]
                    #print(doc_id)
                    if doc_id in all_doc_lengths:
                        length_of_doc_id = int(all_doc_lengths[str(doc_id)])
                    
                        capital_K = k1 * ((1-b) + (b * (length_of_doc_id/avg_length)))
                        score += (float((math.log((D+0.5))/float(df_i+0.5))) * float(((float(1+k1)*tf_d_i)/float(capital_K+tf_d_i))) * float((((1+k2)*float(tf_q_i)/(k2+tf_q_i)))))
                    docs_score_for_each_query[str(doc_id)] = score
                #print(docs_score_for_each_query)
                    
            scores_dictionary[query_id] = dict()
            scores_dictionary[query_id] = docs_score_for_each_query
    return scores_dictionary

In [249]:
calculate_okapi_bm25()

{'202': {},
 '214': {'7116': 4.626100392668382},
 '216': {'6448': 3.307155804139594,
  ' 7116': 3.307155804139594,
  ' 606': 3.307155804139594},
 '221': {'7170': 3.9278961368001486,
  ' 6630': 3.9278961368001486,
  ' 3660': 3.9278961368001486,
  ' 6037': 3.9278961368001486,
  ' 7038': 3.9278961368001486},
 '227': {'2782': 2.953631351311153, ' 6630': 2.953631351311153},
 '230': {},
 '234': {'2037': 8.212477667239865, ' 7455': 8.212477667239865},
 '243': {'5710': 3.769325960104702},
 '246': {},
 '250': {'3660': 3.7417829151391775}}

In [239]:
def dirichlet_smoothing():
    
    

SyntaxError: unexpected EOF while parsing (<ipython-input-239-48b12880a518>, line 1)

In [ ]:
tokensDict=dict()
i=0
file=open("termids.txt",'r',encoding="utf-8")
for value in file:
    value=value.split("/")
    value[1]=value[1].strip("\n")
    tokensDict[str(value[1])]=str(value[0])

    i=i+1
file.close()

###############################
#Average Doc Length
#i=0
#TotalDocLength=0
#file = open("doc_index.txt",'r')
#for value in file:
#    i=i+1
#    print(i)
#    value=value.split("\t")
#    TotalDocLength=TotalDocLength+(len(value)-2)

#averageLength=TotalDocLength/3474
#file.close()
Mubeen
file1 = open("term_index.txt", 'rb')
forwardIndex = {}
queryDocList = []
for i in range(0, len(titles)):
    docsList = []
    vector = []
    for token in queries2[i].keys():
        if token not in vector:
            termid = tokensDict[token]
            forwardIndex[termid]={}
            file2 = open("term_info.txt", 'r')
            for rows in file2:
                rows=rows.split("\t")
                if str(rows[0])==str(termid):
                    file1.seek(int(rows[1]))
                    line=file1.readline().decode()
                    line=line.split("\t")
                    for k in range(1,len(line)):
                        pair=line[k].split(":")
                        if pair[0] not in forwardIndex[termid].keys():
                            forwardIndex[termid][pair[0]]=1

                        else:
                            forwardIndex[termid][pair[0]]=(forwardIndex[termid][pair[0]])+1


                        if pair[0] not in docsList:
                            docsList.append(pair[0])

                    break
            file2.close()
            vector.append(token)
    queryDocList.append(docsList)
file1.close()
Mubeen
file1 = open("term_index.txt", 'rb')
forwardIndex = {}
queryDocList = []
for i in range(0, len(titles)):
    docsList = []
    vector = []
    for token in queries2[i].keys():
        if token not in vector:
            termid = tokensDict[token]
            forwardIndex[termid]={}
            file2 = open("term_info.txt", 'r')
            for rows in file2:
                rows=rows.split("\t")
                if str(rows[0])==str(termid):
                    file1.seek(int(rows[1]))
                    line=file1.readline().decode()
                    line=line.split("\t")
                    for k in range(1,len(line)):
                        pair=line[k].split(":")
                        if pair[0] not in forwardIndex[termid].keys():
                            forwardIndex[termid][pair[0]]=1

                        else:
                            forwardIndex[termid][pair[0]]=(forwardIndex[termid][pair[0]])+1


                        if pair[0] not in docsList:
                            docsList.append(pair[0])

                    break
            file2.close()
            vector.append(token)
    queryDocList.append(docsList)
file1.close()
Mubeen
for i in range(0, len(titles)):
        topicID = str(titles[i].attrs['number'])
        tuples=[]

        for values in docsDict.keys():
            if values in queryDocList[i]:
                vector = []
                docScores = []
                queryScores = []
                doclen=int(getTerms(values))
                for token in queries2[i].keys():
                    if token not in vector:
                        vector.append(token)
                        termid = tokensDict[token]
                        # for query
                        termFreqInQuery = 1
                        score = termFreqInQuery / (termFreqInQuery + 0.5 + (1.5 * (doclen / averageLength)))
                        queryScores.append(score)

                        if values in forwardIndex[termid].keys():
                            termFreqInDocument = forwardIndex[str(termid)][values]
                            score = termFreqInDocument / (termFreqInDocument + 0.5 + (1.5 * (doclen / averageLength)))
                            docScores.append(score)

                        else:
                            docScores.append(0)

                numerator = 0
                for j in range(0, len(docScores)):
                    numerator = numerator + (docScores[j] * queryScores[j])

                docVectorMagnitute = 0
                for j in range(0, len(docScores)):
                        docVectorMagnitute = docVectorMagnitute + (docScores[j] * docScores[j])

                docVectorMagnitute = sqrt(docVectorMagnitute)

                queryVectorMagnitute = 0
                for j in range(0, len(queryScores)):
                    queryVectorMagnitute = queryVectorMagnitute + (queryScores[j] * queryScores[j])

                queryVectorMagnitute = sqrt(queryVectorMagnitute)

                denominator = docVectorMagnitute * queryVectorMagnitute
                if denominator==0:
                    TF=0
                else:
                    TF = numerator / denominator
                pair=[docsDict[values],TF]
                tuples.append(pair)

            else:
                pair=[docsDict[values],0]
                tuples.append(pair)

        for j in range (0,len(tuples)-1):
            min = tuples[j][1]
            for k in range(j+1,len(tuples)):
                if tuples[j][1] < tuples [k][1]:
                    temp=tuples[j]
                    tuples[j]=tuples[k]
                    tuples[k]=temp

        for j in range(0, len(tuples)):
            print(str(topicID) + " 0 " + tuples[j][0] + " " + str(j + 1) + " " + str(tuples[j][1]) + " run1")
Mubeen
#making dictionary of docids DocumentID => [DocumentName]
docsDict={}
file=open("docids.txt",'r')
for value in file:
    value=value.split("/")
    value[1]=value[1].strip("\n")
    docsDict[str(value[0])]=value[1]
file.close()
Mubeen
def getTerms(doc):

    file=open("doc_index.txt")

    i=0
    do=0
    for value in file:
        value=value.split("\t")

        if do == 1 and doc != str(value[0]):
            break

        elif do == 1:
            i=i+(len(value)-2)

        elif doc == str(value[0]):
            do=1
            i = i + (len(value) - 2)

    file.close()
    return i

def getTokenID(token):
    file=open("termids.txt",'r',encoding='utf-8')
    for value in file:
        value=value.split('/')
        value[1]=value[1].strip('\n')
        if(value[1]==token):
            file.close()
            return value[0]

    return -1

def find(big,str2):
    for str in big:
        if (str==str2):
            return 0
    return 1

def termFreqInDoc(doc,term):

    file = open("term_info.txt", 'r')
    offset = 0
    for value in file:
        value = value.split("\t")
        if term == value[0]:
            offset = value[1]

    file.close()

    file = open("term_index.txt", "rb")
    file.seek(int(offset))
    line = file.readline().decode()
    line = line.split("\t")
    freq = 0
    for xx in line:
        xx = xx.split(":")
        if xx[0] == doc:
            freq = freq + 1

    file.close()
    return freq

def getDocID(token):
        file = open("docids.txt", 'r')
        for value in file:
            value = value.split('/')
            value[1] = value[1].strip('\n')
            if (value[1] == token):
                file.close()
                return value[0]
        return -1

def docsContainingTerm(term):
    file = open("term_info.txt", 'r')

    for value in file:
        value = value.split("\t")
        if (value[0] == term):
            return value[3]

    return 0
    file.close()

def termsInCorpus(term):
    file=open("term_info.txt",'r')

    for value in file:
        value=value.split("\t")
        if(value[0]==term):
            return value[2]

    return 0
    file.close()
Mubeen
for i in range(0, len(titles)):
        topicID = str(titles[i].attrs['number'])
        tuples=[]

        for values in docsDict.keys():
            if values in queryDocList[i]:
                vector = []
                docScores = []
                queryScores = []
                doclen=int(getTerms(values))
                for token in queries2[i].keys():
                    if token not in vector:

                        IDF=log10(3474/int(docsContainingTerm(termid)))

                        vector.append(token)
                        termid = tokensDict[token]
                        # for query
                        termFreqInQuery = 1
                        score = termFreqInQuery / (termFreqInQuery + 0.5 + (1.5 * (doclen / averageLength)))
                        score=score*IDF
                        queryScores.append(score)

                        if values in forwardIndex[termid].keys():
                            termFreqInDocument = forwardIndex[str(termid)][values]
                            score = termFreqInDocument / (termFreqInDocument + 0.5 + (1.5 * (doclen / averageLength)))
                            score = score * IDF
                            docScores.append(score)

                        else:
                            docScores.append(0)

                numerator = 0
                for j in range(0, len(docScores)):
                    numerator = numerator + (docScores[j] * queryScores[j])

                docVectorMagnitute = 0
                for j in range(0, len(docScores)):
                        docVectorMagnitute = docVectorMagnitute + (docScores[j] * docScores[j])

                docVectorMagnitute = sqrt(docVectorMagnitute)

                queryVectorMagnitute = 0
                for j in range(0, len(queryScores)):
                    queryVectorMagnitute = queryVectorMagnitute + (queryScores[j] * queryScores[j])

                queryVectorMagnitute = sqrt(queryVectorMagnitute)

                denominator = docVectorMagnitute * queryVectorMagnitute
                if denominator==0:
                    TF=0
                else:
                    TF = numerator / denominator
                pair=[docsDict[values],TF]
                tuples.append(pair)

            else:
                pair=[docsDict[values],0]
                tuples.append(pair)

        for j in range (0,len(tuples)-1):
            min = tuples[j][1]
            for k in range(j+1,len(tuples)):
                if tuples[j][1] < tuples [k][1]:
                    temp=tuples[j]
                    tuples[j]=tuples[k]
                    tuples[k]=temp

        for j in range(0, len(tuples)):
            print(str(topicID) + " 0 " + tuples[j][0] + " " + str(j + 1) + " " + str(tuples[j][1]) + " run1")

#########################################################################################################################################
#########################################################################################################################################
#########################################################################################################################################
######################     Score = Okapi BM25  #######################################################################################
#########################################################################################################################################
elif sys.argv[2]=="BM25":

    k1 = 1.2
    b = 0.75
    k2 = 100

    for i in range(0,len(titles)):
        topicID=titles[i].attrs['number']
        tuples = []

        for values in docsDict.keys():
            BM25 = 0
            if values in queryDocList[i]:

                doclen = getTerms(values)

                K = k1 * ((1 - b) + (b * (int(doclen) / averageLength)))

                for token in queries2[i].keys():

                    termid = tokensDict[token]

                    termFreqInQuery = 1
                    if values in forwardIndex[termid].keys():
                        termFreqInDocument = forwardIndex[termid][values]
                        BM25 = BM25 + log10(        ((doclen + 0.5) / (int(docsContainingTerm(termid)) + 0.5))      *       (((1 + k1) * termFreqInDocument) / (K + termFreqInDocument))        *        ((((1 + k2) * termFreqInQuery) / (k2 + termFreqInQuery)))       )


            pair = [docsDict[values], BM25]
            tuples.append(pair)

            #print(str(topicID) + " 0 " + docsDict[values] + " " + str(j + 1) + " " + str(BM25) + " run1")


        for j in range(0, len(tuples) - 1):
            min = tuples[j][1]
            for k in range(j + 1, len(tuples)):
                if tuples[j][1] < tuples[k][1]:
                    temp = tuples[j]
                    tuples[j] = tuples[k]
                    tuples[k] = temp

        for j in range(0, len(tuples)):
            print(str(topicID) + " 0 " + tuples[j][0] + " " + str(j + 1) + " " + str(tuples[j][1]) + " run1")

#########################################################################################################################################
#########################################################################################################################################
#########################################################################################################################################
######################     Score = Jack Mercer Smoothing   ##############################################################################
#########################################################################################################################################
#########################################################################################################################################
elif sys.argv[2]=="JM":

    for i in range(0,len(titles)):
        topicID=titles[i].attrs['number']
        tuples = []

        lemda=0.2

        for values in docsDict.keys():
            JM = 0
            if values in queryDocList[i]:
                doclen = getTerms(values)

                for token in queries2[i].keys():

                    termid = tokensDict[token]
                    termFreqInQuery = 1

                    if values in forwardIndex[termid].keys():

                        if(JM==0):
                            JM=     (lemda*( forwardIndex[termid][values]))     +       (( 1 - lemda )*(     int(termsInCorpus(termid))/totalDocLength    ))

                        else:
                            JM=JM*(     (lemda*( forwardIndex[termid][values]))     +   (( 1 - lemda )*(     int(termsInCorpus(termid))/totalDocLength    ))    )

            pair=[docsDict[values],JM]
            tuples.append(pair)



        for j in range (0,len(tuples)-1):
            min = tuples[j][1]
            for k in range(j+1,len(tuples)):
                if tuples[j][1] < tuples [k][1]:
                    temp=tuples[j]
                    tuples[j]=tuples[k]
                    tuples[k]=temp


        for j in range(0,len(tuples)):
            print(str(topicID)+" 0 "+tuples[j][0]+" "+str(j+1)+" "+str(tuples[j][1])+" run1")